# Using SavedModel's signature to inlucde preprocessing and postprocessing

This notebook contains the code for my article:

[A Comprehensive Guide to Creating Keras Models including Preprocessing and Postprocessing for Seamless Deployment with TF-Serving using Signatures]()

For an in-depth explanation, please refer to that link.

## Importing the required libraries

In [ ]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
from keras.utils import to_categorical
import numpy as np
from tensorflow.keras import models, layers, Input

## Downloading and preparing the dataset

This part assumes you are using a Linux-compatible OS. If that is not the case, you can either change them to your specific needs to simply ignore them and do the same thing manually.

In [ ]:
!wget https://raw.githubusercontent.com/allisonhorst/palmerpenguins/main/inst/extdata/penguins.csv

The dataset contains a couple of records with NA values. We need to exclude these samples since they will mess up with the schema extraction process.

In [ ]:
# To get rid of the record with NA in them
!grep -Ev "NA" ./penguins.csv > ./penguins_filtered.csv

## Analysing the dataset

Here, we'll be using the Tensorflow data validation library to analyse the dataset and learn more about it.

In [ ]:
examples_file = './penguins_filtered.csv'
dataset_stats = tfdv.generate_statistics_from_csv(examples_file)
tfdv.visualize_statistics(dataset_stats)

The same library can help us identify the type of each feature.

In [ ]:
schema = tfdv.infer_schema(dataset_stats)
tfdv.display_schema(schema=schema)

## Loading and transforming the dataset for training

Before learning how to transforming the dataset the proper way, let's do it in a simple way.

In [ ]:
penguins_ds = tf.data.experimental.make_csv_dataset(
    examples_file,
    batch_size=5,
    label_name='species')

Peeking into the orignal dataset to see some samples

In [ ]:
for element in penguins_ds.take(1):
    print(element)

Applying the transformation to the dataset directly, using the "map" method.

Out of the 7 input features, 6 of them are one-hot encoded and one is normalized. The label feature is also one-hot encoded.

The boundaries for the discretization is calculated using the statistical analysis done above. For all one-hot encodings, 10 buckets are assumed.

In [ ]:
island_lookup = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(
                tf.constant(['Biscoe', 'Dream', 'Torgersen']),
                tf.range(start=0, limit=3, delta=1, dtype=tf.int32)),
            default_value=0)

sex_lookup = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(
                tf.constant(['female', 'male']),
                tf.range(start=0, limit=2, delta=1, dtype=tf.int32)),
            default_value=0)

species_lookup = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(
                tf.constant(['Adelie', 'Chinstrap', 'Gentoo']),
                tf.range(start=0, limit=3, delta=1, dtype=tf.int32)),
            default_value=0)

def calc_bins(start, end, count):
    step = (end - start) / (count)
    return list(np.linspace(start+step, end, count - 1, endpoint=False))

bill_length_mapper = layers.Discretization(bin_boundaries=calc_bins(32.1, 59.6, 10),
                                           output_mode='one_hot')
flapper_length_mapper = layers.Discretization(bin_boundaries=calc_bins(172.0, 231.0, 10),
                                              output_mode='one_hot')
body_mass_mapper = layers.Discretization(bin_boundaries=calc_bins(2700.0, 6300.0, 10),
                                         output_mode='one_hot')


def one_hot_encode(feature, depth):
    feature = tf.squeeze(feature)
    return tf.one_hot(feature, depth=depth)

def transform(features, label):
    return ({'island': one_hot_encode(island_lookup.lookup(features['island']), 3),
             'sex': one_hot_encode(sex_lookup.lookup(features['sex']), 2),
             'bill_length_mm': bill_length_mapper(features['bill_length_mm']),
             'bill_depth_mm': (features['bill_depth_mm'] - 17.16) / 1.97,
             'flipper_length_mm': flapper_length_mapper(features['flipper_length_mm']),
             'body_mass_g': body_mass_mapper(features['body_mass_g']),
             'year': one_hot_encode(features['year'] - 2007, 3)},
            one_hot_encode(species_lookup.lookup(label), 3))

preprocessed_ds = penguins_ds.map(transform)

Let's print and see the result of the transformation

In [ ]:
for element in preprocessed_ds.take(1):
    print(element)

## Designing and training a Keras model

Here, we are instantiating a Keras model using functional API since we have multiple inputs. The design of the model is not really important since this notebook aims at preprocessing and postprocessing, and not trainging a model.

In [ ]:
# Input layers
island_input = Input(batch_shape=(None, 3), name='island')
sex_input = Input(batch_shape=(None, 2), name='sex')
bill_length_mm_input = Input(batch_shape=(None, 10), name='bill_length_mm')
bill_depth_mm_input = Input(batch_shape=(None, 1), name='bill_depth_mm')
flipper_length_mm_input = Input(batch_shape=(None, 10), name='flipper_length_mm')
body_mass_g_input = Input(batch_shape=(None, 10), name='body_mass_g')
year_input = Input(batch_shape=(None, 3), name='year')

# Stack the input layers on top of each other
input_layers = layers.concatenate([island_input,
                                   sex_input,
                                   bill_length_mm_input,
                                   bill_depth_mm_input,
                                   flipper_length_mm_input,
                                   body_mass_g_input, year_input])

# Actual learning layers
x = layers.Dense(128, activation='relu')(input_layers)
output = layers.Dense(3, activation='softmax')(x)

model = models.Model([island_input,
                      sex_input,
                      bill_length_mm_input,
                      bill_depth_mm_input,
                      flipper_length_mm_input,
                      body_mass_g_input, year_input],
                     output)
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy')

Traning the model. This model is an overkill for this task and as the result, the trained model is overfit and cannot generalize well.

In [ ]:
model.fit(preprocessed_ds, epochs=10, steps_per_epoch=1000)

Let's test the trained model and see the generated predictions.

The important note here is that we have to use the preprocessed dataset or the model will throw an error.

In [ ]:
for example in preprocessed_ds.take(1):
    print('\nInput tensors:\n\n', example[0])
    result = model(example[0])
    print('\nOutput tensors:\n\n', example[1])
    print('\nComparing output and target:\n')
    print('Output: ', tf.argmax(result, axis=-1))
    print('Target: ', tf.argmax(example[1], axis=-1))

## Adding the preprocessing and postprocessing layers and saving the model

This class will transform the categorical features into one-hot encoding.

In [ ]:
@tf.keras.utils.register_keras_serializable()
class OneHotEncoder(layers.Layer):
    def __init__(self, inputs: list, **kwargs):
        super(OneHotEncoder, self).__init__(**kwargs)
        self.inputs = inputs

    def build(self, input_shape):
        self.mapping_lookup = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(
                tf.constant(self.inputs),
                tf.range(start=0, limit=len(self.inputs), delta=1, dtype=tf.int32)),
            default_value=0)

    def call(self, input):
        mapped = self.mapping_lookup.lookup(input)
        one_hot = tf.one_hot(mapped, depth=len(self.inputs))
        return tf.reshape(one_hot, [-1, len(self.inputs)])

    def get_config(self):
        config = super().get_config()
        config['inputs'] = self.inputs
        return config

This class is designed to transform the numberical features into one-hot encoding.

In [ ]:
@tf.keras.utils.register_keras_serializable()
class OneHotDiscretization(layers.Layer):
    def __init__(self, start, end, count, **kwargs):
        super(OneHotDiscretization, self).__init__(**kwargs)
        self.start = start
        self.end = end
        self.count = count

    def build(self, input_shape):
        self.bin_boundaries = self.calc_bins(self.start, self.end, self.count)
        self.disc = tf.keras.layers.Discretization(bin_boundaries=self.bin_boundaries,
                                                   output_mode='one_hot')

    def call(self, input):
        return self.disc(input)

    def calc_bins(self, start, end, count):
        step = (end - start) / (count)
        return list(np.linspace(start+step, end, count - 1, endpoint=False))

    def get_config(self):
        config = super().get_config()
        config['start'] = self.start
        config['end'] = self.end
        config['count'] = self.count
        return config

This class will take in one-hot encoding and transform them into original values.

In [ ]:
@tf.keras.utils.register_keras_serializable()
class OneHotDecoder(layers.Layer):
    def __init__(self, outputs: list, **kwargs):
        super(OneHotDecoder, self).__init__(**kwargs)
        self.outputs = outputs

    def build(self, input_shape):
        self.mapping_lookup = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(
                tf.range(start=0, limit=len(self.outputs), delta=1, dtype=tf.int32),
                tf.constant(self.outputs)),
            default_value=self.outputs[0])

    def call(self, one_hot):
        index = tf.math.argmax(one_hot, axis=-1)
        prediction = self.mapping_lookup.lookup(tf.cast(index, dtype=tf.int32))
        return prediction

    def get_config(self):
        config = super().get_config()
        config['outputs'] = self.outputs
        return config

These three functions will create a signature which add preprocessing and postprocessing to the model.

In [ ]:
def preprocessing_fn():
    """Transforming the dataset's structure to model's expectation"""

    island_input = Input(shape=(1,),
                         name='island',
                         dtype=tf.string)
    island_encoder = OneHotEncoder(['Biscoe',
                                    'Dream',
                                    'Torgersen'])(island_input)

    sex_input = Input(shape=(1,),
                      name='sex',
                      dtype=tf.string)
    sex_encoder = OneHotEncoder(['female',
                                 'male'])(sex_input)

    bill_length_input = Input(shape=(1,),
                              name='bill_length_mm',
                              dtype=tf.float32)
    bill_length_mapper = OneHotDiscretization(32.1,
                                              59.6,
                                              10)(bill_length_input)
    
    bill_depth_input = Input(shape=(1,),
                             name='bill_depth_mm',
                             dtype=tf.float32)
    bill_depth_mapper = (bill_depth_input - 17.16) / 1.97
    
    flipper_length_input = Input(shape=(1,),
                                 name='flipper_length_mm',
                                 dtype=tf.int32)
    flipper_length_mapper = OneHotDiscretization(172.0,
                                                 231.0,
                                                 10)(flipper_length_input)
    
    body_mass_input = Input(shape=(1,),
                            name='body_mass_g',
                            dtype=tf.int32)
    body_mass_mapper = OneHotDiscretization(2700.0,
                                            6300.0,
                                            10)(body_mass_input)
    
    year_input = Input(shape=(1,),
                       name='year',
                       dtype=tf.int32)
    year_mapper = tf.reshape(tf.one_hot(year_input - 2007,
                                        depth=3), [-1, 3])

    return models.Model([island_input,
                         sex_input,
                         bill_length_input,
                         bill_depth_input,
                         flipper_length_input,
                         body_mass_input,
                         year_input],
                        [island_encoder,
                         sex_encoder,
                         bill_length_mapper,
                         bill_depth_mapper,
                         flipper_length_mapper,
                         body_mass_mapper,
                         year_mapper])


def postprocessing_fn():
    species_input = Input(shape=(3,), name='species', dtype=tf.float32)
    species_lookup = OneHotDecoder(['Adelie',
                                    'Chinstrap',
                                    'Gentoo'])(species_input)
    
    return models.Model([species_input], [species_lookup])


def serving_default_fn(model):
    model.preprocessing_layer = preprocessing_fn()
    model.postprocessing_layer = postprocessing_fn()
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None,],
                                                dtype=tf.string,
                                                name='island'),
                                  tf.TensorSpec(shape=[None,],
                                                dtype=tf.string,
                                                name='sex'),
                                  tf.TensorSpec(shape=[None,],
                                                dtype=tf.float32,
                                                name='bill_length_mm'),
                                  tf.TensorSpec(shape=[None,],
                                                dtype=tf.float32,
                                                name='bill_depth_mm'),
                                  tf.TensorSpec(shape=[None,],
                                                dtype=tf.int32,
                                                name='flipper_length_mm'),
                                  tf.TensorSpec(shape=[None,],
                                                dtype=tf.int32,
                                                name='body_mass_g'),
                                  tf.TensorSpec(shape=[None,],
                                                dtype=tf.int32,
                                                name='year')])
    def _serving_default_fn(island, sex,
                            bill_length_mm,
                            bill_depth_mm,
                            flipper_length_mm,
                            body_mass_g,
                            year):
        """Composing an end-to-end model, including preprocessing and postprocessing"""

        inputs = {'island': island,
                  'sex': sex,
                  'bill_length_mm': bill_length_mm,
                  'bill_depth_mm': bill_depth_mm,
                  'flipper_length_mm': flipper_length_mm,
                  'body_mass_g': body_mass_g,
                  'year': year}
        
        transformed = model.preprocessing_layer(inputs)
        output = model(transformed)
        return model.postprocessing_layer(output)
    
    return _serving_default_fn

Saving the model and a signature which includes the proprocessing and postprocessing

In [ ]:
model.save('palmer_penguins', save_format='tf', signatures={'serving_default': serving_default_fn(model)})

Demonstrating how the three layers can work together to transform the features, generate predictions, and finally, transoform the output to human-readable values.

In [ ]:
for example in penguins_ds.take(1):
    transformed = model.preprocessing_layer(example[0])
    output = model(transformed)
    result = model.postprocessing_layer(output)
    print('\nOutput:\n')
    print(result)
    print('\nTarget:\n')
    print(example[1])

## Loading the saved model

Loading the saved model into a new object for another demonstration.

At this point, you can restart the kernel and see that model definition is not needed for the following code to work.

In [ ]:
reconstructed_model = tf.keras.models.load_model("palmer_penguins")
list(reconstructed_model.signatures.keys())

Using the signature for inference.

The signature cannot be called with positional arguments, it needs to be called by named arguments. And this is by design.

In [ ]:
sig = reconstructed_model.signatures['serving_default']

for example in penguins_ds.take(1):
    result = sig(island = example[0]['island'],
                 sex = example[0]['sex'],
                 bill_length_mm = example[0]['bill_length_mm'],
                 bill_depth_mm = example[0]['bill_depth_mm'],
                 flipper_length_mm = example[0]['flipper_length_mm'],
                 body_mass_g = example[0]['body_mass_g'],
                 year = example[0]['year'])
    print('\nInput:\n\n', example[0])
    print('\nOutput:\n\n', result)
    print('\nTarget:\n\n', example[1])

## Deploying the model to TF-Serving

In order to feed the saved model to a TF-Serving instance, we need change the folder structure first.

In [ ]:
!mkdir ./palmer_penguins/2

In [ ]:
!mv ./palmer_penguins/assets \
    ./palmer_penguins/fingerprint.pb \
    ./palmer_penguins/keras_metadata.pb \
    ./palmer_penguins/saved_model.pb  \
    ./palmer_penguins/variables \
    ./palmer_penguins/2/

Starting a docker to run a TF-Serving instance. You might need to change the folder path.

```shell
$ docker run -t --rm -p 8501:8501 \
    -v "/home/mehran/penguins/palmer_penguins:/models/palmer_penguins" \
    -e MODEL_NAME=palmer_penguins \
    --name "palmer_penguins" \
    tensorflow/serving
```

Then, you can send a request to the server like this:

```shell
$ curl --location --request POST 'http://127.0.0.1:8501/v1/models/palmer_penguins:predict' \
--header 'Content-Type: application/json' \
--data-raw '{
    "instances": [
        {
            "island": "Dream",
            "bill_length_mm": 39.5,
            "bill_depth_mm": 16.7,
            "flipper_length_mm": 178,
            "body_mass_g": 3250,
            "sex": "female",
            "year": 2007
        },
        {
            "island": "Biscoe",
            "bill_length_mm": 44.4,
            "bill_depth_mm": 17.3,
            "flipper_length_mm": 219,
            "body_mass_g": 5250,
            "sex": "male",
            "year": 2008
        }
    ]
}'
```

Which will output:

```json
{
    "predictions": ["Adelie", "Gentoo"]
}
```

Once you are done with TF-Serving, you can stop it:

```shell
$ docker stop palmer_penguins
```